# SQL

[SQL](https://en.wikipedia.org/wiki/SQL) is a domain-specific language used in programming and designed for managing data held in a relational database. 


A relational database is a database that stores related information across multiple tables and allows you to query information in more than one table at the same time. 

Within a table, the data to be stored is organized in a tabular format with rows and columns. Each row inside a table represents a distinct record with the column headings specifying the corresponding type of data stored. S

There are many DB systems available like MySQL, PostgreSQL, Oracle Database (don't use this), MS SQL Server, Amazon Redshift, etc. However, all of them speak SQL, so once you've got the hang of the basic SQL syntax you'll be able to work with any of them.

To keep things simple, we will be using a database system called [SQLite](https://www.sqlite.org/about.html) to practice SQL syntax. 

Unlike most other SQL databases, SQLite does not have a separate server. SQLite reads and writes directly to ordinary files on your computer. A complete SQL database with multiple tables, etc., is contained in a single file.

### Resources

Since SQL is the main database language used worldwide, there are plenty of resources to master it:

- https://sqlzoo.net/

- https://sqlbolt.com

- leetcode.com

- https://www.sql-ex.ru/learn_exercises.php

# Using Raw SQL cursor connections

In [1]:
import sqlite3
conn = sqlite3.connect('data/mtcars.sqlite')
c = conn.cursor()

In [2]:
# create and execute our query
cylinders = 4
cursor = c.execute(f"SELECT * FROM results WHERE cylinders = {cylinders}")

In [3]:
# fetch a list of the column names
column_names = list(map(lambda x: x[0], cursor.description))

print(column_names)

# fetch all rows and map column names onto them
rows = list(map(lambda x: dict(zip(column_names, x)), cursor.fetchall()))

# print the name and MPG for each result
for row in rows:
    print("{name}:\t{mpg}".format(**row))

['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'year', 'origin', 'name']
toyota corona mark ii:	24.0
datsun pl510:	27.0
volkswagen 1131 deluxe sedan:	26.0
peugeot 504:	25.0
audi 100 ls:	24.0
saab 99e:	25.0
bmw 2002:	26.0
datsun pl510:	27.0
chevrolet vega 2300:	28.0
toyota corona:	25.0
ford pinto:	25.0
chevrolet vega (sw):	22.0
mercury capri 2000:	23.0
opel 1900:	28.0
peugeot 304:	30.0
fiat 124b:	30.0
toyota corolla 1200:	31.0
datsun 1200:	35.0
volkswagen model 111:	27.0
plymouth cricket:	26.0
toyota corona hardtop:	24.0
dodge colt hardtop:	25.0
volkswagen type 3:	23.0
chevrolet vega:	20.0
ford pinto runabout:	21.0
volvo 145e (sw):	18.0
volkswagen 411 (sw):	22.0
peugeot 504 (sw):	21.0
renault 12 (sw):	26.0
ford pinto (sw):	22.0
datsun 510 (sw):	28.0
toyouta corona mark ii (sw):	23.0
dodge colt (sw):	28.0
toyota corolla 1600 (sw):	27.0
volkswagen super beetle:	26.0
toyota carina:	20.0
chevrolet vega:	21.0
datsun 610:	22.0
ford pinto:	19.0
fiat 124 sport coup

# Using SQL from pandas

In [4]:
import pandas as pd
df = pd.read_sql_query("SELECT * from results", conn)
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130,3504.0,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693.0,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150,3436.0,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150,3433.0,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140,3449.0,10.5,70,1,ford torino


### Simple query

In [5]:
pd.read_sql_query(
"""
SELECT  
    mpg, 
    displacement,
    name
FROM results
--- this is a SQL code comment
WHERE year > 75
    and acceleration < 18
""", 
    con=conn
)

,mpg,displacement,name
0,28.0,107.0,fiat 131
1,25.0,116.0,opel 1900
2,25.0,140.0,capri ii
3,26.0,98.0,dodge colt
4,27.0,101.0,renault 12tl
...,...,...,...
165,32.0,144.0,toyota celica gt
166,36.0,135.0,dodge charger 2.2
167,27.0,151.0,chevrolet camaro
168,27.0,140.0,ford mustang gl


### more complex query using Groupbys

In [6]:
pd.read_sql(
"""
SELECT  
    AVG(mpg) as avg_mpg,
    SUM(mpg) as sum_mpg,
    AVG(horsepower) as hp,
    AVG(acceleration)
FROM results
WHERE year > 75
GROUP BY cylinders
ORDER BY avg_mpg DESC
""", 
    con=conn
)

,avg_mpg,sum_mpg,hp,AVG(acceleration)
0,31.446565,4119.5,74.809160,16.551145
1,27.366667,82.1,82.333333,18.633333
2,22.600000,45.2,105.000000,13.000000
3,21.440000,964.8,103.977778,16.224444
4,17.270588,587.2,140.382353,14.144118


### Making a new table

In [11]:
df.to_sql(
    name="test_output",
    con=conn, 
    schema=None, 
    if_exists='replace', 
    index=True, 
)

In [12]:
### Note the LIMIT operator. It's the equivalent of df.head(10)
pd.read_sql("select * from test_output limit 10", con=conn)

,index,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,0,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu
1,1,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320
2,2,18.0,8,318.0,150.0,3436.0,11.0,70,1,plymouth satellite
3,3,16.0,8,304.0,150.0,3433.0,12.0,70,1,amc rebel sst
4,4,17.0,8,302.0,140.0,3449.0,10.5,70,1,ford torino
5,5,15.0,8,429.0,198.0,4341.0,10.0,70,1,ford galaxie 500
6,6,14.0,8,454.0,220.0,4354.0,9.0,70,1,chevrolet impala
7,7,14.0,8,440.0,215.0,4312.0,8.5,70,1,plymouth fury iii
8,8,14.0,8,455.0,225.0,4425.0,10.0,70,1,pontiac catalina
9,9,15.0,8,390.0,190.0,3850.0,8.5,70,1,amc ambassador dpl


In [13]:
# pd.read_sql(
# """
# create table test_2 (
#     mpg float,
#     cylinders int,
#     displacement float,
#     horsepower float,
#     weight float,
#     acceleration float,
#     year int,
#     origin int,
#     name varchar(256)
# );
# """, con=conn)

### List  tables in a database

Table and index names can be list by doing a **SELECT** on a special table named "***SQLITE_MASTER***". Every SQLite database has an SQLITE_MASTER table that defines the schema for the database. For tables, the ***type*** field will always be '***table***' and the name field will be the name of the table. So to get a list of all tables in the database, use the following SELECT command:

See more at https://www.sqlite.org/faq.html#q7.

In [ ]:
pd.read_sql("SELECT name FROM sqlite_master WHERE type='table'", con=conn)

# Making a new DB

Is as simple as connecting to a new file name

In [2]:
conn = sqlite3.connect('test.sqlite')